In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import hh
import importlib
importlib.reload(hh)
from ipywidgets import *
%matplotlib widget

## Simulating the HH experience

In [ ]:
neuron = hh.HH(tFinal=80)
neuron.initialize()
print(f'number of timepoints = {len(neuron.time)}')
arr_Iext = np.zeros(neuron.time.shape[0])
arr_Iext[10:] = -6

dt_input = widgets.BoundedFloatText(value=0.1, min=0.01, max=1.0, step=0.01, description='dt', disabled=False)
button = widgets.Button(description='Run', disabled=False, button_style='', tooltip='Run', icon='check')

display(button)
display(dt_input)
fig, axs = plt.subplots(2, 1, figsize=(8, 4))
axs[0].plot(neuron.time, arr_Iext)
axs[0].set_ylabel('Input current')
ax = axs[1]
ax.set_xlim(0, neuron.time.max())
ax.set_ylim(-20, 100)
ax.scatter(0, -neuron.arr_V[0], s=5, c='k')
ax.set_ylabel('Membrane potential')

i = 0
def on_button_clicked(b):
    global i
    global neuron
    global arr_Iext
    for iteration in range(500):
        neuron.dt = dt_input.value
        Ik, Ina, Il = neuron.update_currents(i)
        neuron.arr_V[i+1] = neuron.update_voltage(Ik, Ina, Il, arr_Iext[i], i)
        ax.scatter(neuron.time[i], -neuron.arr_V[i], s=5, c='C4')
        i+=1
        fig.canvas.draw_idle()

button.on_click(on_button_clicked)


In [ ]:
# What the above graph should look like
%matplotlib inline
neuron = hh.HH(tFinal=80)
neuron.initialize()
neuron.simulate(arr_Iext)
plt.figure()
plt.plot(neuron.time, -neuron.arr_V, 'k')
plt.show()
%matplotlib widget

#### "The step value had to be very small initially (since there are no differences at t = 0) and it also had to be changed repeatedly during a run, because the differences became unmanageable if it was too large. It varied between about 0.01 ms at the beginning of a run or 0.02 ms during the rising phase of the action potential, and 1 ms during the small oscillations which follow the spike."

Hodgkin & Huxley

## Simulate constant current inputs of varying amplitudes

In [ ]:
# Precompute outputs
ls_Iexts = []
ls_arr_outs = []
for Iamp in np.arange(-3, -14, -0.25):
    neuron = hh.HH(tFinal=100)
    neuron.initialize()
    
    Iext = np.zeros(neuron.time.shape[0])
    Iext[100:] = Iamp
    
    neuron.simulate(Iext)
    ls_Iexts.append(Iext)

    arr_append = np.dstack((neuron.arr_h, neuron.arr_m, neuron.arr_n, neuron.arr_V))
    ls_arr_outs.append(arr_append)

In [ ]:
# Interactive plot
idx = 0
arr_out = ls_arr_outs[idx].squeeze()
Iext = ls_Iexts[idx]
fig = plt.figure(figsize=(10, 6))
ax1 = fig.add_subplot(3, 1, 1)
l1 = ax1.plot(Iext)
ax1.set_ylim(-15, 10)
ax1.set_title(f'Input current')# Title doesn't dynamically update  = {Iext.max():.2f}')

ax = fig.add_subplot(3, 1, 2)
l2 = ax.plot(-arr_out[:, 3])
ax.set_ylim(-20, 100)
ax.set_title('Membrane potential')

ax = fig.add_subplot(3, 1, 3)
l3 = ax.plot(arr_out[:, 0], label='h')
l4 = ax.plot(arr_out[:, 1], label='m')
l5 = ax.plot(arr_out[:, 2], label='n')
ax.legend(loc=(1.05, 0.5))
ax.set_ylim(0, 1)
plt.show()
plt.subplots_adjust(hspace=0.5)

def update(idx=0):
    arr_out = ls_arr_outs[idx].squeeze()
    Iext = ls_Iexts[idx]
    l1[0].set_ydata(Iext)
    l2[0].set_ydata(-arr_out[:, 3])
    l3[0].set_ydata(arr_out[:, 0])
    l4[0].set_ydata(arr_out[:, 1])
    l5[0].set_ydata(arr_out[:, 2])
    # ax1.set_title(f'Input current = {Iext.max():.2f}')
    fig.canvas.draw_idle()

interact(update, idx=(0, len(ls_Iexts)-1, 1))

## Why do we see the sudden spiking behavior?

## What do you notice about the refractory period?